In [1]:
import re
import time
import pickle
import numpy as np

from edit_distance import SequenceMatcher
import torch
from dataset import SpeechDataset
import matplotlib.pyplot as plt

from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.model import GRUDecoder
import pickle
import argparse
import matplotlib.pyplot as plt
from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.lm_utils import build_llama_1B
from neural_decoder.model import GRUDecoder
from neural_decoder.bit import BiT_Phoneme
import pickle
import argparse
from lm_utils import _cer_and_wer
import json
import os

/home3/ebrahim2/miniconda3/envs/speech-bci/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def convert_sentence(s):
    s = s.lower()
    charMarks = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 "'", ' ']
    ans = []
    for i in s:
        if(i in charMarks):
            ans.append(i)
    
    return ''.join(ans)

In [10]:
base_dir = "/home3/skaasyap/willett"

load_lm = True
# LM decoding hyperparameters
acoustic_scale = 0.8
blank_penalty = np.log(2)

run_for_llm = False

if run_for_llm:
    return_n_best = True
    rescore = False
    nbest = 100
    print("RUNNING IN LLM MODE")
else:
    return_n_best = False
    rescore = False
    nbest = 1
    print("RUNNING IN N-GRAM MODE")
    
device = "cuda:1"

RUNNING IN N-GRAM MODE


In [4]:
if load_lm: 
        
    lmDir = base_dir +'/lm/languageModel'
    ngramDecoder = lmDecoderUtils.build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")
    

loaded LM


I0510 17:50:31.338227 1316189 brain_speech_decoder.h:52] Reading fst /home3/skaasyap/willett/lm/languageModel/TLG.fst
I0510 17:53:10.544240 1316189 brain_speech_decoder.h:81] Reading symbol table /home3/skaasyap/willett/lm/languageModel/words.txt


# Get model predictions
## Always check to make sure val perf matches wandb

In [39]:
output_file = 'obi'

if output_file == 'obi':
    model_storage_path = '/data/willett_data/outputs/'
elif output_file == 'leia':
    model_storage_path = '/data/willett_data/leia_outputs/'
    
models_to_run = ['gru_held_out_days_2_redo', 'gru_held_out_days_1_redo', 'gru_held_out_days_redo']

shared_output_file = 'gru_held_out_days'

if len(shared_output_file) > 0:
    print("Writing to shared output file")
    write_mode = "a"
else:
    write_mode = "w"
    
seeds_list = [0,1,2,3]
partition = "competition" # "test"
run_lm = True

fill_max_day = True


if partition == 'test':
    saveFolder_transcripts = "/data/willett_data/model_transcriptions/"
else:
    saveFolder_transcripts = "/data/willett_data/model_transcriptions_comp/"
    
for seed in seeds_list:
    
    for model_name_str in models_to_run:
        
        modelPath = f"{model_storage_path}{model_name_str}_seed_{seed}"
        
        if len(shared_output_file) > 0:
            output_file = f"{shared_output_file}_seed_{seed}"
            print(output_file)
        else:
            output_file = f"{model_name_str}_seed_{seed}"
            
        with open(modelPath + "/args", "rb") as handle:
            args = pickle.load(handle)
            
        if args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both':
            data_file = '/data/willett_data/ptDecoder_ctc_both'
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data':
            data_file = '/data/willett_data/ptDecoder_ctc'
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both_held_out_days':
            data_file = '/data/willett_data/ptDecoder_ctc_both_held_out_days'
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both_held_out_days_1':
            data_file = '/data/willett_data/ptDecoder_ctc_both_held_out_days_1'
            
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both_held_out_days_2':
            data_file = '/data/willett_data/ptDecoder_ctc_both_held_out_days_2'
            
        else:
            data_file = args['datasetPath']
            
        trainLoaders, testLoaders, loadedData = getDatasetLoaders(
            data_file, 8
        )
        
        # if true, model is a GRU
        if 'nInputFeatures' in args.keys():
            
            if 'max_mask_pct' not in args:
                args['max_mask_pct'] = 0
            if 'num_masks' not in args:
                args['num_masks'] = 0
            if 'input_dropout' not in args:
                args['input_dropout'] = 0
                
            print("Loading GRU")
            model = GRUDecoder(
                neural_dim=args["nInputFeatures"],
                n_classes=args["nClasses"],
                hidden_dim=args["nUnits"],
                layer_dim=args["nLayers"],
                nDays=args['nDays'],
                dropout=args["dropout"],
                device=device,
                strideLen=args["strideLen"],
                kernelLen=args["kernelLen"],
                gaussianSmoothWidth=args["gaussianSmoothWidth"],
                bidirectional=args["bidirectional"],
                input_dropout=args['input_dropout'], 
                max_mask_pct=args['max_mask_pct'],
                num_masks=args['num_masks']
            ).to(device)

        else:
            
            if 'mask_token_zero' not in args:
                args['mask_token_zero'] = False
            
            print("Loading TRANSFORMER")
            
            # Instantiate model
            model = BiT_Phoneme(
                patch_size=args['patch_size'],
                dim=args['dim'],
                dim_head=args['dim_head'],
                nClasses=args['nClasses'],
                depth=args['depth'],
                heads=args['heads'],
                mlp_dim_ratio=args['mlp_dim_ratio'],
                dropout=args['dropout'],
                input_dropout=args['input_dropout'],
                look_ahead=args['look_ahead'],
                gaussianSmoothWidth=args['gaussianSmoothWidth'],
                T5_style_pos=args['T5_style_pos'],
                max_mask_pct=args['max_mask_pct'],
                num_masks=args['num_masks'], 
                mask_token_zeros=args['mask_token_zero'], 
                num_masks_channels=0, 
                max_mask_channels=0, 
                dist_dict_path=0, 
            ).to(device)
            
            
        ckpt_path = modelPath + '/modelWeights'
        model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
        model = model.to(device)
        
        model.eval()

        model_outputs = {
            "logits": [],
            "logitLengths": [],
            "trueSeqs": [],
            "transcriptions": [],
        }
        
        total_edit_distance = 0
        total_seq_length = 0

        if partition == "competition":
            
            if fill_max_day:
                testDayIdxs = np.arange(5)
            else:
                testDayIdxs = [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]
                
        elif partition == "test":
            
            testDayIdxs = range(len(loadedData[partition])) 
            
        ground_truth_sentences = []
        
        print("RESTRICTED DAYS: ", args['restricted_days'])
        with torch.no_grad():
            
            for i, testDayIdx in enumerate(testDayIdxs):
                
                if len(args['restricted_days']) > 0:
                    if testDayIdx not in args['restricted_days']:
                        continue
                    
                test_ds = SpeechDataset([loadedData[partition][i]])
                test_loader = torch.utils.data.DataLoader(
                    test_ds, batch_size=1, shuffle=False, num_workers=0
                )
                
                
                for j, (X, y, X_len, y_len, _) in enumerate(test_loader):
                            
                    X, y, X_len, y_len, dayIdx = (
                        X.to(device),
                        y.to(device),
                        X_len.to(device),
                        y_len.to(device),
                        torch.tensor([testDayIdx], dtype=torch.int64).to(device),
                    )
                    
                    if fill_max_day:
                        dayIdx.fill_(args['maxDay'])
                                                
                    pred = model.forward(X, X_len, dayIdx)
                    
                    if hasattr(model, 'compute_length'):
                        adjustedLens = model.compute_length(X_len)
                    else:
                        adjustedLens = ((X_len - model.kernelLen) / model.strideLen).to(torch.int32)
                        
                    for iterIdx in range(pred.shape[0]):
                        
                        trueSeq = np.array(y[iterIdx][0 : y_len[iterIdx]].cpu().detach())
                        model_outputs["logits"].append(pred[iterIdx].cpu().detach().numpy())
                        
                        model_outputs["logitLengths"].append(
                            adjustedLens[iterIdx].cpu().detach().item()
                        )
                        
                        model_outputs["trueSeqs"].append(trueSeq)
                        
                        decodedSeq = torch.argmax(
                            torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),
                            dim=-1,
                        ) 
                        
                        decodedSeq = torch.unique_consecutive(decodedSeq, dim=-1)
                        decodedSeq = decodedSeq.cpu().detach().numpy()
                        decodedSeq = np.array([i for i in decodedSeq if i != 0])
                        
                        matcher = SequenceMatcher(
                            a=trueSeq.tolist(), b=decodedSeq.tolist()
                        )
                        
                        total_edit_distance += matcher.distance()
                        total_seq_length += len(trueSeq)
                        
                    transcript = loadedData[partition][i]["transcriptions"][j].strip()
                    transcript = re.sub(r"[^a-zA-Z\- \']", "", transcript)
                    transcript = transcript.replace("--", "").lower()
                    model_outputs["transcriptions"].append(transcript)

            cer = total_edit_distance / total_seq_length
            
            print("Model performance: ", cer)

            if run_lm:
                
                llm_outputs = []
                start_t = time.time()
                nbest_outputs = []
                
                for j in range(len(model_outputs["logits"])):
                    
                    logits = model_outputs["logits"][j]
                    
                    logits = np.concatenate(
                        [logits[:, 1:], logits[:, 0:1]], axis=-1
                    )  # Blank is last token
                    
                    logits = lmDecoderUtils.rearrange_speech_logits(logits[None, :, :], has_sil=True)
                    
                    nbest = lmDecoderUtils.lm_decode(
                        ngramDecoder,
                        logits[0],
                        blankPenalty=blank_penalty,
                        returnNBest=return_n_best,
                        rescore=rescore,
                    )
                    
                    nbest_outputs.append(nbest)
                    
                time_per_sample = (time.time() - start_t) / len(model_outputs["logits"])
                print(f"N-gram decoding took {time_per_sample} seconds per sample")
                
                if run_for_llm:
                    print("SAVING OUTPUTS FOR LLM")
                    with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_model_outputs.pkl", "wb") as f:
                        pickle.dump(model_outputs, f)
                        
                    with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_nbest.pkl", "wb") as f:
                        pickle.dump(nbest_outputs, f)
                    
                else:
                    # just get perf with greedy decoding
                    for i in range(len(model_outputs["transcriptions"])):
                        model_outputs["transcriptions"][i] = model_outputs["transcriptions"][i].strip()
                        nbest_outputs[i] = nbest_outputs[i].strip()
                    
                    # lower case + remove puncs
                    for i in range(len(model_outputs["transcriptions"])):
                        model_outputs["transcriptions"][i] = convert_sentence(model_outputs["transcriptions"][i])

                    cer, wer = _cer_and_wer(nbest_outputs, model_outputs["transcriptions"], 
                                        outputType='speech', returnCI=True)

                    print("CER and WER after 3-gram LM: ", cer, wer)       
                    
                    out_file = os.path.join(saveFolder_transcripts, output_file)   # no extension per your spec
                    
                    with open(out_file + '.txt', write_mode, encoding="utf-8") as f:
                        f.write("\n".join(nbest_outputs)+ "\n")   # one line per LLM output  

Writing to shared output file
gru_held_out_days_seed_0
Loading GRU
RESTRICTED DAYS:  []


/tmp/ipykernel_1316189/3668896128.py:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


Model performance:  2.218125
gru_held_out_days_seed_0
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.8053125
gru_held_out_days_seed_0
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.853125
gru_held_out_days_seed_1
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.1496875
gru_held_out_days_seed_1
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.8084375
gru_held_out_days_seed_1
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.8425
gru_held_out_days_seed_2
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.201875
gru_held_out_days_seed_2
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.7828125
gru_held_out_days_seed_2
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.8540625
gru_held_out_days_seed_3
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.2075
gru_held_out_days_seed_3
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.773125
gru_held_out_days_seed_3
Loading GRU
RESTRICTED DAYS:  []
Model performance:  2.87625
